<a href="https://colab.research.google.com/github/BF667/codename-rvc-fork-4/blob/main/assets/notebooks/Codename_RVC_Fork_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Mount Drive
from google.colab import drive
from google.colab._message import MessageError

try:
  drive.mount("/content/drive")
except MessageError:
  print("❌ Failed to mount drive")

In [ ]:

#@title Setup runtime environment
from IPython.display import clear_output
import codecs


repo_name = "codename-rvc-fork-4"
url = "https://github.com/BF667/codename-rvc-fork-4"
!git config --global advice.detachedHead false
!pip install uv -q
!git clone $url
%cd codename-rvc-fork-4


LOGS_PATH = f"/content/{repo_name}/logs"
BACKUPS_PATH = f"/content/drive/MyDrive/{repo_name}-Backup"


print("Installing requirements...")
!curl -LsSf https://astral.sh/uv/install.sh | sh
!uv pip install -q -r requirements_cloud.txt
!uv pip install -q ngrok jupyter-ui-poll
!npm install -g -q localtunnel &> /dev/null
clear_output()
print("✅ Finished installing requirements!")

In [ ]:
# @title Sync with Google Drive
# @markdown <small> 💾 Run this cell to automatically Save/Load models from your mounted drive
from IPython.display import display, clear_output
from pathlib import Path

non_bak_folders = ["mute", "reference", "zips", "mute_spin"]
non_bak_path = "/tmp/rvc_logs"


def press_button(button):
  button.disabled = True


def get_date(path: Path):
  from datetime import datetime
  return datetime.fromtimestamp(int(path.stat().st_mtime))


def get_size(path: Path):
  !du -shx --apparent-size "{path}" > /tmp/size.txt
  return open("/tmp/size.txt").readlines().pop(0).split("   ")[0] + "B"


def sync_folders(folder: Path, backup: Path):
  from ipywidgets import widgets
  from jupyter_ui_poll import ui_events
  from time import sleep

  local = widgets.VBox([
      widgets.Label(f"Local: {LOGS_PATH.removeprefix('/content/')}/{folder.name}/"),
      widgets.Label(f"Size: {get_size(folder)}"),
      widgets.Label(f"Last modified: {get_date(folder)}")
  ])
  remote = widgets.VBox([
      widgets.Label(f"Remote: {BACKUPS_PATH.removeprefix('/content/')}/{backup.name}/"),
      widgets.Label(f"Size: {get_size(backup)}"),
      widgets.Label(f"Last modified: {get_date(backup)}")
  ])
  separator = widgets.VBox([
      widgets.Label("|||"),
      widgets.Label("|||"),
      widgets.Label("|||")
  ])
  radio = widgets.RadioButtons(
      options=[
          "Save local model to drive",
          "Keep remote model"
      ]
  )
  button = widgets.Button(
      description="Sync",
      icon="upload",
      tooltip="Sync model"
  )
  button.on_click(press_button)

  clear_output()
  print(f"Your local model '{folder.name}' is in conflict with it's copy in Google Drive.")
  print("Please select which one you want to keep:")
  display(widgets.Box([local, separator, remote]))
  display(radio)
  display(button)

  with ui_events() as poll:
    while not button.disabled:
      poll(10)
      sleep(0.1)

  match radio.value:
    case "Save local model to drive":
      !rm -r "{backup}"
      !mv "{folder}" "{backup}"
    case "Keep remote model":
      !rm -r "{folder}"


if Path("/content/drive").is_mount():
  !mkdir -p "{BACKUPS_PATH}"
  !mkdir -p "{non_bak_path}"

  if not Path(LOGS_PATH).is_symlink():
    for folder in non_bak_folders:
      folder = Path(f"{LOGS_PATH}/{folder}")
      backup = Path(f"{BACKUPS_PATH}/{folder.name}")

      !mkdir -p "{folder}"
      !mv "{folder}" "{non_bak_path}" &> /dev/null
      !rm -rf "{folder}"
      folder = Path(f"{non_bak_path}/{folder.name}")
      if backup.exists() and backup.resolve() != folder.resolve():
        !rm -r "{backup}"
      !ln -s "{folder}" "{backup}" &> /dev/null

    for model in Path(LOGS_PATH).iterdir():
      if model.is_dir() and not model.is_symlink():
        backup = Path(f"{BACKUPS_PATH}/{model.name}")

        if model.name == ".ipynb_checkpoints":
          continue

        if backup.exists() and backup.is_dir():
          sync_folders(model, backup)
        else:
          !rm "{backup}"
          !mv "{model}" "{backup}"

    !rm -r "{LOGS_PATH}"
    !ln -s "{BACKUPS_PATH}" "{LOGS_PATH}"

    clear_output()
    print("✅ Models are synced!")

  else:
    !rm "{LOGS_PATH}"
    !ln -s "{BACKUPS_PATH}" "{LOGS_PATH}"
    clear_output()
    print("✅ Models already synced!")

else:
  print("❌ Drive is not mounted, skipping model syncing")
  print("To sync your models, first mount your Google Drive and re-run this cell")

In [ ]:
# @title Run
import os
import shutil
import codecs
import time
import threading
import urllib.request
import ipywidgets as widgets
print('requirements installed successfully! Launching ssh and interface...')
print("Starting localtunnel...")
!npm install -g localtunnel
with open('url.txt', 'w') as file:
    file.write('')
get_ipython().system_raw('lt --port 7897 >> url.txt 2>&1 &')
time.sleep(3)
try:
   endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
   with open('url.txt', 'r') as file:
        tunnel_url = file.read().replace("your url is: ", "").strip()
   print(f"Share Link: \033[93m{tunnel_url}\033[0m")
   print(f"Password IP: {endpoint_ip}")
   !python app.py
except Exception as e:
    print(f"Error starting localtunnel: {e}")